In [1]:
using JuMP, CSV, DataFrames, Gurobi, Interpolations

In [2]:
# Read the test and train CSV files
data_targeted = CSV.File("/Users/sanyachauhan/Downloads/Data_Targeted.csv") |> DataFrame;
data_email = CSV.File("/Users/sanyachauhan/Downloads/Data_Emails.csv") |> DataFrame;
data_print = CSV.File("/Users/sanyachauhan/Downloads/Data_Print.csv") |> DataFrame;
data_influencer = CSV.File("/Users/sanyachauhan/Downloads/Data_Influencers.csv") |> DataFrame;

In [3]:
B = [1000.0, 1500.0, 2000.0];

In [4]:
# Creating linear interpolation functions for each marketing option
interp_targeted = LinearInterpolation(data_targeted[!, :Cost], data_targeted[!, :Views]);
interp_email = LinearInterpolation(data_email[!, :Cost], data_email[!, :Views]);
interp_print = LinearInterpolation(data_print[!, :Cost], data_print[!, :Views]);
interp_influencers = LinearInterpolation(data_influencer[!, :Cost], data_influencer[!, :Views]);

In summary, linear interpolation is used here to create a continuous function from your discrete data points, allowing the optimization model to work effectively with any investment amount within the provided ranges. It bridges the gap between your data and the needs of the optimization solver.

In [5]:
for Budget in B
        # Optimization model
        model = Model(Gurobi.Optimizer)
        
        # Decision variables for investments
        @variable(model, investment_targeted >= 0)
        @variable(model, investment_email >= 0)
        @variable(model, investment_print >= 0)
        @variable(model, investment_influencers >= 0)
        
        # Additional variables and constraints for piecewise linear segments for each marketing option
        # Helps in modeling non-linear relationships
        
        # Creating a piecewise linear function
        # Targeted Online Advertising
        @variable(model, targeted_segments[1:3] >= 0) #each col is non-negative
        
        breakpoints_targeted = [0, 40, 310, 990] 
        #defines the investment levels at which the slope (rate of increase in views) changes. 
        #These breakpoints divide the investment range into segments.
        
        slopes_targeted = [8, 10, 12] 
        # represents the rate of change in views per unit of investment in each segment. For instance, a slope of 0.8 for the 
        #first segment means that for each unit of investment in this segment, the views increase by 0.8.
        
        @constraint(model, investment_targeted == sum(targeted_segments)) #the sum of investments across all segments
        
        @constraint(model, sum(targeted_segments[i] for i in 1:3) <= breakpoints_targeted[end])
        for i in 1:3
            @constraint(model, targeted_segments[i] <= breakpoints_targeted[i+1] - breakpoints_targeted[i])
        end
        
        # Email Marketing - done
        # Example breakpoints and slopes for Email Marketing
        @variable(model, email_segments[1:3] >= 0) 
        breakpoints_email = [0, 77.05, 144.87, 500] # Example breakpoints 
        slopes_email = [18, 19.99, 19.95] # Example slopes
        @constraint(model, investment_email == sum(email_segments))
        @constraint(model, sum(email_segments[i] for i in 1:3) <= breakpoints_email[end])
        for i in 1:3
            @constraint(model, email_segments[i] <= breakpoints_email[i+1] - breakpoints_email[i])
        end
        
        # Print Media - done
        # Example breakpoints and slopes for Print Media
        @variable(model, print_segments[1:3] >= 0)
        slopes_print = [12, 40, 73] # Example breakpoints
        breakpoints_print = [200, 375, 565, 695] # Example slopes
        @constraint(model, investment_print == sum(print_segments))
        @constraint(model, sum(print_segments[i] for i in 1:3) <= breakpoints_print[end])
        for i in 1:3
            @constraint(model, print_segments[i] <= breakpoints_print[i+1] - breakpoints_print[i])
        end
        
        # Influencer Marketing - done
        # Example breakpoints and slopes for Influencer Marketing
        @variable(model, influencers_segments[1:3] >= 0)
        breakpoints_influencers = [100, 298, 406, 694] # Example breakpoints
        slopes_influencers = [10, 16, 12] # Example slopes
        @constraint(model, investment_influencers == sum(influencers_segments))
        @constraint(model, sum(influencers_segments[i] for i in 1:3) <= breakpoints_influencers[end])
        for i in 1:3
            @constraint(model, influencers_segments[i] <= breakpoints_influencers[i+1] - breakpoints_influencers[i])
        end
        
        # Objective: Maximize views
        @objective(model, Max, sum(slopes_targeted[i] * targeted_segments[i] for i in eachindex(slopes_targeted)) + 
                      sum(slopes_email[i] * email_segments[i] for i in eachindex(slopes_email)) +
                      sum(slopes_print[i] * print_segments[i] for i in eachindex(slopes_print)) +
                      sum(slopes_influencers[i] * influencers_segments[i] for i in eachindex(slopes_influencers)))

        
        # Budget constraint
            @constraint(model, Budget, investment_targeted + investment_email +
                           investment_print + investment_influencers <= Budget)
        
        # Solve the model
        optimize!(model)
        
        # Results
        println("Optimal Investments with Budget = ", Budget)
        println("Targeted Online Advertising: \$", value(investment_targeted))
        println("Email Marketing: \$", value(investment_email))
        println("Print Media: \$", value(investment_print))
        println("Influencer Marketing: \$", value(investment_influencers))
        println("Total Views: ", objective_value(model))

        # Save the model in LP format with a dynamic name
        model_filename = "model1_" * string(Budget) * ".lp"
        write_to_file(model, model_filename)
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-31
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 21 rows, 16 columns and 44 nonzeros
Model fingerprint: 0x942af4f9
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+00, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 1e+03]
Presolve removed 21 rows and 16 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9509465e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.950946530e+04

User-callback calls 59, time in user-callback 0.00 sec
Optimal Investments with Budget = Budget : investment_targeted + investment_email + investment_print + investment_influence